In [37]:
import numpy as np

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, Lambda
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential

In [38]:
text = 'Deep learning is a subset of machine learning that focuses on utilizing neural networks to perform tasks such as classification, regression, and representation learning. The field takes inspiration from biological neuroscience and is centered around stacking artificial neurons into layers and "training" them to process data. The adjective "deep" refers to the use of multiple layers (ranging from three to several hundred or thousands) in the network. Methods used can be either supervised, semi-supervised or unsupervised. Some common deep learning network architectures include fully connected networks, deep belief networks, recurrent neural networks, convolutional neural networks, generative adversarial networks, transformers, and neural radiance fields. These architectures have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board gamef programs, where they have produced results comparable to and in some cases surpassing human expert performance. Early forms of neural networks were inspired by information processing and distributed communication nodes in biological systems, particularly the human brain. However, current neural networks do not intend to model the brain function of organisms, and are generally seen as low-quality models for that purpose.'

# Preprocessing Text

In [39]:
# Splitting the text into sentences

sentences = text.split('.')
sentences = [sentence.lower() for sentence in sentences]
sentences

['deep learning is a subset of machine learning that focuses on utilizing neural networks to perform tasks such as classification, regression, and representation learning',
 ' the field takes inspiration from biological neuroscience and is centered around stacking artificial neurons into layers and "training" them to process data',
 ' the adjective "deep" refers to the use of multiple layers (ranging from three to several hundred or thousands) in the network',
 ' methods used can be either supervised, semi-supervised or unsupervised',
 ' some common deep learning network architectures include fully connected networks, deep belief networks, recurrent neural networks, convolutional neural networks, generative adversarial networks, transformers, and neural radiance fields',
 ' these architectures have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, 

In [40]:
# Tokenizing the sentences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)   # learns the vocabulary from the given sentences, creating an internal mapping of each unique word to an integer index
sequences = tokenizer.texts_to_sequences(sentences) # Each word in the sentences is replaced by its corresponding index from the tokenizer's vocabulary.
print(sequences)

[[6, 7, 10, 27, 28, 8, 11, 7, 12, 29, 30, 31, 4, 1, 3, 32, 33, 34, 13, 35, 36, 2, 37, 7], [5, 38, 39, 40, 14, 15, 41, 2, 10, 42, 43, 44, 45, 46, 47, 16, 2, 48, 49, 3, 50, 51], [5, 52, 6, 53, 3, 5, 54, 8, 55, 16, 56, 14, 57, 3, 58, 59, 17, 60, 9, 5, 18], [61, 62, 63, 64, 65, 19, 66, 19, 17, 67], [20, 68, 6, 7, 18, 21, 69, 70, 71, 1, 6, 72, 1, 73, 4, 1, 74, 4, 1, 75, 76, 1, 77, 2, 4, 78, 22], [79, 21, 23, 80, 81, 3, 22, 82, 83, 84, 85, 86, 87, 88, 24, 11, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 2, 100, 101, 102, 103, 104, 23, 105, 106, 107, 3, 2, 9, 20, 108, 109, 25, 110, 111], [112, 113, 8, 4, 1, 114, 115, 116, 117, 24, 2, 118, 119, 120, 9, 15, 121, 122, 5, 25, 26], [123, 124, 4, 1, 125, 126, 127, 3, 128, 5, 26, 129, 8, 130, 2, 131, 132, 133, 13, 134, 135, 136, 137, 12, 138], []]


In [41]:
idx2word = tokenizer.index_word
word2idx = tokenizer.word_index

print(idx2word, '\n')
print(word2idx)

{1: 'networks', 2: 'and', 3: 'to', 4: 'neural', 5: 'the', 6: 'deep', 7: 'learning', 8: 'of', 9: 'in', 10: 'is', 11: 'machine', 12: 'that', 13: 'as', 14: 'from', 15: 'biological', 16: 'layers', 17: 'or', 18: 'network', 19: 'supervised', 20: 'some', 21: 'architectures', 22: 'fields', 23: 'have', 24: 'processing', 25: 'human', 26: 'brain', 27: 'a', 28: 'subset', 29: 'focuses', 30: 'on', 31: 'utilizing', 32: 'perform', 33: 'tasks', 34: 'such', 35: 'classification', 36: 'regression', 37: 'representation', 38: 'field', 39: 'takes', 40: 'inspiration', 41: 'neuroscience', 42: 'centered', 43: 'around', 44: 'stacking', 45: 'artificial', 46: 'neurons', 47: 'into', 48: 'training', 49: 'them', 50: 'process', 51: 'data', 52: 'adjective', 53: 'refers', 54: 'use', 55: 'multiple', 56: 'ranging', 57: 'three', 58: 'several', 59: 'hundred', 60: 'thousands', 61: 'methods', 62: 'used', 63: 'can', 64: 'be', 65: 'either', 66: 'semi', 67: 'unsupervised', 68: 'common', 69: 'include', 70: 'fully', 71: 'connected

# Creating Contexts and Integers

In [42]:
vocab_size = len(tokenizer.word_index) + 1
emb_size = 100
context_size = 2

contexts = []
targets = []

for sequence in sequences:
    for i in range(context_size, len(sequence) - context_size):
        target = sequence[i]
        context = [sequence[i - 2], sequence[i - 1], sequence[i + 1], sequence[i + 2]]
        contexts.append(context)
        targets.append(target)

print(contexts, "\n")
print(targets)

[[6, 7, 27, 28], [7, 10, 28, 8], [10, 27, 8, 11], [27, 28, 11, 7], [28, 8, 7, 12], [8, 11, 12, 29], [11, 7, 29, 30], [7, 12, 30, 31], [12, 29, 31, 4], [29, 30, 4, 1], [30, 31, 1, 3], [31, 4, 3, 32], [4, 1, 32, 33], [1, 3, 33, 34], [3, 32, 34, 13], [32, 33, 13, 35], [33, 34, 35, 36], [34, 13, 36, 2], [13, 35, 2, 37], [35, 36, 37, 7], [5, 38, 40, 14], [38, 39, 14, 15], [39, 40, 15, 41], [40, 14, 41, 2], [14, 15, 2, 10], [15, 41, 10, 42], [41, 2, 42, 43], [2, 10, 43, 44], [10, 42, 44, 45], [42, 43, 45, 46], [43, 44, 46, 47], [44, 45, 47, 16], [45, 46, 16, 2], [46, 47, 2, 48], [47, 16, 48, 49], [16, 2, 49, 3], [2, 48, 3, 50], [48, 49, 50, 51], [5, 52, 53, 3], [52, 6, 3, 5], [6, 53, 5, 54], [53, 3, 54, 8], [3, 5, 8, 55], [5, 54, 55, 16], [54, 8, 16, 56], [8, 55, 56, 14], [55, 16, 14, 57], [16, 56, 57, 3], [56, 14, 3, 58], [14, 57, 58, 59], [57, 3, 59, 17], [3, 58, 17, 60], [58, 59, 60, 9], [59, 17, 9, 5], [17, 60, 5, 18], [61, 62, 64, 65], [62, 63, 65, 19], [63, 64, 19, 66], [64, 65, 66, 19

In [43]:
# Printing features with target
for i in range(5):
    words = []
    target = idx2word.get(targets[i])
    for j in contexts[i]:
        words.append(idx2word.get(j))
    print(words," -> ", target)

['deep', 'learning', 'a', 'subset']  ->  is
['learning', 'is', 'subset', 'of']  ->  a
['is', 'a', 'of', 'machine']  ->  subset
['a', 'subset', 'machine', 'learning']  ->  of
['subset', 'of', 'learning', 'that']  ->  machine


# Creating Training and Testing Data

In [44]:
X = np.array(contexts)
y = np.array(targets)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Defining Model

In [45]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=emb_size, input_length=2 * context_size),
    Lambda(lambda x: tf.reduce_mean(x, axis=1)),
    Dense(256, activation='relu'),
    Dense(512, activation='relu'),
    Dense(vocab_size, activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [46]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20)

Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step - accuracy: 0.0224 - loss: 4.9328 - val_accuracy: 0.0000e+00 - val_loss: 4.9372
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0858 - loss: 4.9095 - val_accuracy: 0.0000e+00 - val_loss: 4.9430
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.0950 - loss: 4.8759 - val_accuracy: 0.0000e+00 - val_loss: 4.9543
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.0950 - loss: 4.8216 - val_accuracy: 0.0000e+00 - val_loss: 4.9790
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0763 - loss: 4.7404 - val_accuracy: 0.0000e+00 - val_loss: 5.0327
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0773 - loss: 4.6167 - val_accuracy: 0.0000e+00 - val_loss: 5.1550
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.0427 - loss: 4.4181 - val_accuracy: 0.0000e+00 - val_loss: 5.4481
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0635 - loss: 4.1985 - val_accuracy

# Output

In [47]:
test_words=[idx2word[index] for index in contexts[0]]

input_data=np.expand_dims(contexts[0],axis=0)
print(input_data)

pred=model.predict(input_data)
predicted_index=np.argmax(pred[0])

print("Context words:", test_words)
print("Predicted terget word : ",idx2word[predicted_index])

[[ 6  7 27 28]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
Context words: ['deep', 'learning', 'a', 'subset']
Predicted terget word :  is
